# Sentiment Analysis Twitter Tweets

## Motivation/State of the Art
### Motivation
Die Analyse von Stimmungen in sozialen Medien hat in den letzten Jahren an Bedeutung gewonnen. Unternehmen, Forscher und Organisationen verwenden Sentimentanalyse, um Einblicke in die Meinungen der Öffentlichkeit zu gewinnen. Dieses Projekt konzentriert sich auf die Sentimentanalyse von Twitter-Tweets und nutzt Methoden des Natural Language Processing (NLP) um Tweets als positiv oder negativ zu klassifizieren.

Das Ziel ist es, verschiedene Modelle und Architekturen zu vergleichen, die in der Sentimentanalyse verwendet werden, um die beste Leistung hinsichtlich Genauigkeit und Robustheit zu ermitteln. Das Projekt ist inspiriert von bestehenden Arbeiten in der Stimmungsanalyse, die Techniken wie LSTM, Vorverarbeitung von Textdaten und Explainable AI (z. B. LIME) einsetzen.

### State of the Art

Transformer-basierte Modelle (z. B. BERT, RoBERTa): Transformer-Modelle dominieren die NLP-Landschaft mit herausragender Leistung bei Textklassifikationsaufgaben.

Rekurrente neuronale Netze (RNNs) und LSTM: Diese Modelle können den sequentiellen Kontext von Texten erfassen und sind besonders nützlich für Tweets, die oft aufeinander aufbauen.

Explainable AI (LIME): LIME erklärt die Vorhersagen von Modellen durch die Analyse der Bedeutung einzelner Textmerkmale (z. B. Wörter oder Phrasen).

# Importieren von Bibliotheken
Importieren wir die benötigten Bibliotheken.

In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger_eng')

from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob
import re
import string
import tensorflow
from sklearn.metrics  import classification_report ,confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


# Daten einlesen und vorbereiten
Wir verwenden ein vorhandenes Twitter-Datenset, das Tweets und ihre Labels (0 = negativ, 4 = positiv) enthält:

In [6]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", engine="python")
data.columns = ["label", "id", "date", "query", "username", "text"]

In [7]:
data.head()

,label,id,date,query,username,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


die Spalten auf "text" und "label" reduzieren

In [9]:
data=data[['text','label']]

In [10]:
data.head()

,text,label
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0


In [11]:
data.columns

Index(['text', 'label'], dtype='object')

In [12]:
print('data length:', len(data))

data length: 1599999


In [13]:
data.shape

(1599999, 2)

In [14]:
data.info

<bound method DataFrame.info of                                                       text  label
0        is upset that he can't update his Facebook by ...      0
1        @Kenichan I dived many times for the ball. Man...      0
2          my whole body feels itchy and like its on fire       0
3        @nationwideclass no, it's not behaving at all....      0
4                            @Kwesidei not the whole crew       0
...                                                    ...    ...
1599994  Just woke up. Having no school is the best fee...      4
1599995  TheWDB.com - Very cool to hear old Walt interv...      4
1599996  Are you ready for your MoJo Makeover? Ask me f...      4
1599997  Happy 38th Birthday to my boo of alll time!!! ...      4
1599998  happy #charitytuesday @theNSPCC @SparksCharity...      4

[1599999 rows x 2 columns]>

In [15]:
data.dtypes

text     object
label     int64
dtype: object

Das Label der Datensätze der Positivität von "4" auf "1" ändern

In [17]:
data.loc[data['label']==4, 'label'] = 1

In [18]:
data.info

<bound method DataFrame.info of                                                       text  label
0        is upset that he can't update his Facebook by ...      0
1        @Kenichan I dived many times for the ball. Man...      0
2          my whole body feels itchy and like its on fire       0
3        @nationwideclass no, it's not behaving at all....      0
4                            @Kwesidei not the whole crew       0
...                                                    ...    ...
1599994  Just woke up. Having no school is the best fee...      1
1599995  TheWDB.com - Very cool to hear old Walt interv...      1
1599996  Are you ready for your MoJo Makeover? Ask me f...      1
1599997  Happy 38th Birthday to my boo of alll time!!! ...      1
1599998  happy #charitytuesday @theNSPCC @SparksCharity...      1

[1599999 rows x 2 columns]>

In [19]:
data["label"].value_counts()

label
1    800000
0    799999
Name: count, dtype: int64

In [20]:
data_pos = data[data['label'] == 1]
data_neg = data[data['label'] == 0]

Datensatz reduzieren da er für unsere Hardware zu anspruchsvoll ist.

Das ursprüngliche Datenset ist sehr groß (1,6 Millionen Zeilen), was den Entwicklungsprozess verlangsamen würde. Eine kleinere Stichprobe reicht aus, um den Workflow zu testen.

In [22]:
data_pos = data_pos.iloc[:int(100000)]
data_neg = data_neg.iloc[:int(100000)]

In [23]:
data = pd.concat([data_pos, data_neg])
data["label"].value_counts()
del data_pos
del data_neg

# Data Preprocessing (Text Cleaning)
Tweets enthalten oft Rauschen wie URLs, Benutzernamen, Sonderzeichen und Abkürzungen. Diese Informationen stören das Modelltraining und müssen entfernt werden.

Wie:
1. Abkürzungen ausschreiben
2. alles in Lowercase umwandeln
3. Stopwords aus den Texten entfernen
4. URLs, Benutzernamen und Sonderzeichen aus den Texten entfernen
5. Lemmatization
6. Daten tokenisieren & in Sequenzen umwandeln
7. In Trainings/Validierungs & Test-Datenset aufteilen

In [25]:
data.info

<bound method DataFrame.info of                                                      text  label
799999       I LOVE @Health4UandPets u guys r the best!!       1
800000  im meeting up with one of my besties tonight! ...      1
800001  @DaRealSunisaKim Thanks for the Twitter add, S...      1
800002  Being sick can be really cheap when it hurts t...      1
800003    @LovesBrooklyn2 he has that effect on everyone       1
...                                                   ...    ...
99995   looks like my routers broke  more tweets from ...      0
99996   i really dont want to be in college right now....      0
99997                         @flossa  *offers you pepto*      0
99998   @JosieHobo I WOULD SOOOOO BE THERE IF I DIDN'T...      0
99999   OMG, I just moisturised and my legs are BURNIN...      0

[200000 rows x 2 columns]>

# Abkürzungen ausschreiben
Da viele Twitter User Abkürzungen verwenden werden diese in ihre eigentlichen vollständigen Wörter umgewandelt 

In [27]:
chat_word = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now',
    'B4N': 'Bye For Now',
    'BRB': 'Be Right Back',
    'BRT': 'Be Right There',
    'BTW': 'By The Way',
    'B4': 'Before',
    'CU': 'See You',
    'CUL8R': 'See You Later',
    'CYA': 'See You',
    'FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed',
    'FWIW': "For What It's Worth",
    'FYI': 'For Your Information',
    'GAL': 'Get A Life',
    'GG': 'Good Game',
    'GN': 'Good Night',
    'GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!',
    'G9': 'Genius',
    'IC': 'I See',
    'ICQ': 'I Seek you (also a chat program)',
    'ILU': 'ILU: I Love You',
    'IMHO': 'In My Honest/Humble Opinion',
    'IMO': 'In My Opinion',
    'IOW': 'In Other Words',
    'IRL': 'In Real Life',
    'KISS': 'Keep It Simple, Stupid',
    'LDR': 'Long Distance Relationship',
    'LMAO': 'Laugh My A.. Off',
    'LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See',
    'L8R': 'Later',
    'MTE': 'My Thoughts Exactly',
    'M8': 'Mate',
    'NRN': 'No Reply Necessary',
    'OIC': 'Oh I See',
    'PITA': 'Pain In The A..',
    'PRT': 'Party',
    'PRW': 'Parents Are Watching',
    'QPSA?': 'Que Pasa?',
    'ROFL': 'Rolling On The Floor Laughing',
    'ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My A.. Off',
    'SK8': 'Skate',
    'STATS': 'Your sex and age',
    'ASL': 'Age, Sex, Location',
    'THX': 'Thank You',
    'TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later',
    'R': 'ARE',
    'U': 'You',
    'U2': 'You Too',
    'U4E': 'Yours For Ever',
    'WB': 'Welcome Back',
    'WTF': 'What The F...',
    'WTG': 'Way To Go!',
    'WUF': 'Where Are You From?',
    'W8': 'Wait...',
    '7K': 'Sick:-D Laugher',
    'TFW': 'That feeling when',
    'MFW': 'My face when',
    'MRW': 'My reaction when',
    'IFYP': 'I feel your pain',
    'TNTL': 'Trying not to laugh',
    'JK': 'Just kidding',
    'IDC': "I don't care",
    'ILY': 'I love you',
    'IMU': 'I miss you',
    'ADIH': 'Another day in hell',
    'ZZZ': 'Sleeping, bored, tired',
    'WYWH': 'Wish you were here',
    'TIME': 'Tears in my eyes',
    'BAE': 'Before anyone else',
    'FIMH': 'Forever in my heart',
    'BSAAW': 'Big smile and a wink',
    'BWL': 'Bursting with laughter',
    'BFF': 'Best friends forever',
    'CSL': "Can't stop laughing"
}

In [28]:
def short_conv(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_word:
            new_text.append(chat_word[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)
data['text'] = data['text'].apply(lambda text: short_conv(text))
data['text'].head()

799999      I LOVE @Health4UandPets You guys ARE the best!!
800000    im meeting up with one of my besties tonight! ...
800001    @DaRealSunisaKim Thanks for the Twitter add, S...
800002    Being sick can be really cheap when it hurts t...
800003       @LovesBrooklyn2 he has that effect on everyone
Name: text, dtype: object

# Spelling Correction
Viele User vertippen sich oft, um mehr Konsistenz zu haben wird dies korrigiert.
(Auskommentiert, da es extrem viel Leistung zieht und lange braucht)

In [30]:
#data['text'] = data['text'].apply(lambda text: TextBlob(text).correct().string)

In [31]:
#data['text'].head()

# Text in Lowercase umwandeln
Um Konsistenz zu gewährleisten.

In [33]:
data['text']=data['text'].str.lower()
data['text'].head()

799999      i love @health4uandpets you guys are the best!!
800000    im meeting up with one of my besties tonight! ...
800001    @darealsunisakim thanks for the twitter add, s...
800002    being sick can be really cheap when it hurts t...
800003       @lovesbrooklyn2 he has that effect on everyone
Name: text, dtype: object

# Stopwords entfernen
Stopword sind Wörter wie "and", "the" oder "is" und tragen oft keine Bedeutung zur Sentimentanalyse bei. Daher werden sie entfernt.

In [35]:
stopwords_list = stopwords.words('english')
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [36]:
STOPWORDS = set(stopwords.words('english'))
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
data['text'] = data['text'].apply(lambda text: cleaning_stopwords(text))
data['text'].head()

799999                    love @health4uandpets guys best!!
800000    im meeting one besties tonight! cant wait!! - ...
800001    @darealsunisakim thanks twitter add, sunisa! g...
800002    sick really cheap hurts much eat real food plu...
800003                      @lovesbrooklyn2 effect everyone
Name: text, dtype: object

# URLs, Benutzernamen, Sonderzeichen und Zahlen entfernen
Tweets enthalten oft Links, Erwähnungen, Zahlen oder Sonderzeichen die für uns irrelevant sind.

In [38]:
urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern = r"@[^\s]+"
#hastagPattern = r"#[^\s]+"
alphaPattern = r"[^a-zA-Z]"

In [39]:
data['text'] = data['text'].apply(lambda text: re.sub(userPattern,'', text))
#data['text'] = data['text'].apply(lambda text: re.sub(hastagPattern,'',text))
data['text'] = data['text'].apply(lambda text: re.sub(urlPattern,'',text))
data['text'] = data['text'].apply(lambda text: re.sub(alphaPattern,' ', text))
data['text'].head()

799999                                    love  guys best  
800000    im meeting one besties tonight  cant wait     ...
800001     thanks twitter add  sunisa  got meet hin show...
800002    sick really cheap hurts much eat real food plu...
800003                                      effect everyone
Name: text, dtype: object

Sonderzeichen entfernen

In [41]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [42]:
data['text']= data['text'].apply(lambda x: cleaning_punctuations(x))
data['text'].head()

799999                                    love  guys best  
800000    im meeting one besties tonight  cant wait     ...
800001     thanks twitter add  sunisa  got meet hin show...
800002    sick really cheap hurts much eat real food plu...
800003                                      effect everyone
Name: text, dtype: object

# Sequenzen von mehr als 3 gleichen Buchstaben kürzen
Sequenzen von mehr als 3 gleichen Buchstaben sind oft Tippfehler oder es wird "Slang" verwendet. Für mehr Konsistenz werden Sequenzen von mehr als 3 Buchstaben auf 2 gekürzt.

In [44]:
sequencePattern = r"(.)\1\1+"
seqReplacePattern = r"\1\1"
def cleaning_repeating_char(text):
    return re.sub(sequencePattern, seqReplacePattern, text)

In [45]:
data['text'] = data['text'].apply(lambda x: cleaning_repeating_char(x))
data['text'].head()

799999                                    love  guys best  
800000    im meeting one besties tonight  cant wait  gir...
800001     thanks twitter add  sunisa  got meet hin show...
800002    sick really cheap hurts much eat real food plu...
800003                                      effect everyone
Name: text, dtype: object

# Sätze in Listen umwandeln

In [47]:
tokenizer = RegexpTokenizer(r'\w+')
data['text'] = data['text'].apply(tokenizer.tokenize)
data['text'].head()

799999                                   [love, guys, best]
800000    [im, meeting, one, besties, tonight, cant, wai...
800001    [thanks, twitter, add, sunisa, got, meet, hin,...
800002    [sick, really, cheap, hurts, much, eat, real, ...
800003                                   [effect, everyone]
Name: text, dtype: object

# STEMMING
Wörter werden auf "Stammformen" gekürzt. (Wird im Projekt nicht verwendet, da wir uns für Lemmatization entschieden haben)

In [49]:
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data

#data['text']= data['text'].apply(lambda x: stemming_on_text(x))

# LEMMATIZATION
Wörter werden in ihre Grundform zurückgeführt (z. B. "running" → "run").

In [51]:

# Funktion zur Konvertierung von NLTK-POS-Tags zu WordNet-Tags
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)  # Standardmäßig Substantiv

lm = WordNetLemmatizer()

def lemmatizer_on_text(data):
    text = [lm.lemmatize(word, pos=get_wordnet_pos(word)) for word in data]
    return text

data['text'] = data['text'].apply(lambda x: lemmatizer_on_text(x))
data['text'].head()

799999                                    [love, guy, best]
800000    [im, meeting, one, besties, tonight, cant, wai...
800001    [thanks, twitter, add, sunisa, get, meet, hin,...
800002    [sick, really, cheap, hurt, much, eat, real, f...
800003                                   [effect, everyone]
Name: text, dtype: object

# Daten vorbereiten
Daten für das Modell in numerische Sequenzen umwandeln

In [53]:
X=data.text
y=data.label

### Tokenizer
Wir verwenden den Tokenizer, um Wörter in numerische Indizes zu kodieren, und padden die Sequenzen auf eine einheitliche Länge.

In [55]:
max_len = 500
tok = Tokenizer(num_words=5000)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
sequences_matrix.shape

(200000, 500)

### Datenset in Training/Validation und Test Datenset aufteilen (80/20)

In [57]:
X_train, X_test, Y_train, Y_test = train_test_split(sequences_matrix, y, test_size=0.2, random_state=2)

# Model
Das LSTM-Modell ist speziell für sequentielle Daten wie Text geeignet. Es kann sich Kontextinformationen aus früheren Wörtern merken.
Das Modell besteht aus mehreren Schichten:

### Embedding-Schicht
Die Embedding-Schicht wandelt Wörter in dichte, numerische Vektoren um, die semantische Informationen enthalten.
Diese Transformation hilft, die Beziehungen zwischen Wörtern (z. B. Synonyme oder ähnliche Kontexte) mathematisch darzustellen.
Parameter:
input_dim=5000: Die Anzahl der häufigsten Wörter im Vokabular.
output_dim=75: Die Dimension der Vektorrepräsentation für jedes Wort.

### LSTM-Schichten
LSTM-Schichten ermöglichen es dem Modell, sich an frühere Informationen in einer Sequenz zu erinnern und dabei irrelevante Details zu "vergessen".

Tweets sind sequenzielle Daten, bei denen der Kontext (z. B. die Bedeutung eines Wortes basierend auf vorherigen Wörtern) entscheidend ist.

Parameter:
units=64: Die Anzahl der Einheiten (Neuronen) in der LSTM-Schicht.
recurrent_dropout=0.2: Dropout für rekurrente Verbindungen (zwischen den Zeitschritten der Sequenz).
dropout=0.5: Dropout für Verbindungen zwischen den Neuronen in der Schicht.

### Dense-Schicht
Eine vollständig verbundene Schicht, die die gelernten Merkmale in ein repräsentatives Format umwandelt, bevor sie zur Klassifikation verwendet werden. Diese Schicht ermöglicht eine zusätzliche nicht-lineare Transformation der Daten.

### Batch-Normalisierung
Normiert die Ausgabe der vorherigen Schicht, um den Lernprozess zu stabilisieren und zu beschleunigen.

### Dropout-Schichten
Deaktiviert während des Trainings zufällig eine bestimmte Anzahl von Neuronen in einer Schicht.
Dropout hilft, Overfitting zu vermeiden, indem das Modell gezwungen wird, robuste Merkmale zu lernen, die nicht von einer bestimmten Teilmenge von Neuronen abhängen.

### Ausgabeschicht (Dense)
Eine Sigmoid-Aktivierungsfunktion wird verwendet, um die Wahrscheinlichkeiten für die binäre Klassifikation (positiv/negativ) auszugeben. Die Sigmoid-Funktion beschränkt die Ausgabe auf einen Bereich zwischen 0 und 1, was ideal für binäre Klassifikationen ist.


Optimizer: Adam (Adaptive Moment Estimation) für schnelle Konvergenz.

Verlustfunktion: Binary Crossentropy, geeignet für binäre Klassifikationsaufgaben.

Metrik: Genauigkeit (Accuracy), um die Leistung des Modells zu bewerten.

In [60]:

model = Sequential()
model.add(Embedding(5000,75,input_length=max_len))

# Add the first LSTM layer
model.add(LSTM(64, return_sequences=True,recurrent_dropout=0.2,dropout=0.4))  # Return sequences to pass to the next LSTM layer
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
# Add a third LSTM layer
model.add(LSTM(64,recurrent_dropout=0.3,dropout=0.3))
model.add(Dropout(0.4))

# Add a fully connected layer
model.add(Dense(64, activation='relu'))  # Intermediate dense layer for more learning capacity
model.add(BatchNormalization())
model.add(Dropout(0.4))  # Dropout for dense layer

# Output layer
model.add(Dense(1, activation='sigmoid'))  # For classification

# Compile the model
model.compile(
    loss='binary_crossentropy',  # Use sparse if labels are integers
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

C:\Users\alexa\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Callbacks
Das Modell lernt aus den Trainingsdaten, während die Validierungsdaten dazu dienen, Overfitting zu verhindern.
Wir verwenden EarlyStopping, um das Training zu beenden, wenn die Validierungsleistung nicht mehr besser wird und verwenden ReduceLROnPlateau um die Lernrate sinken zu lassen, falls wir uns auf einem Plateau befinden.

In [62]:
checkpoint = EarlyStopping(monitor='val_accuracy',patience = 3 ,mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001)
callbacks_lst = [checkpoint,reduce_lr]

# MODEL TRAINING
mit einer Batchsize von 128 und 15 Epochen. Aus dem Trainingsdatenset werden 20% für die Validierung reserviert und dann verwendet.

In [64]:
print("Training on GPU...") if tensorflow.config.list_physical_devices('GPU') else print("Training on CPU...")

Training on CPU...


In [ ]:
history = model.fit(X_train,Y_train,batch_size=128,epochs=15, validation_split=0.2, callbacks=callbacks_lst)

Epoch 1/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1083s 1s/step - accuracy: 0.6560 - loss: 0.6013 - val_accuracy: 0.7592 - val_loss: 0.4925 - learning_rate: 0.0010
Epoch 2/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1128s 1s/step - accuracy: 0.7705 - loss: 0.4884 - val_accuracy: 0.7678 - val_loss: 0.4819 - learning_rate: 0.0010
Epoch 3/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1134s 1s/step - accuracy: 0.7786 - loss: 0.4716 - val_accuracy: 0.7688 - val_loss: 0.4786 - learning_rate: 0.0010
Epoch 4/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1135s 1s/step - accuracy: 0.7862 - loss: 0.4584 - val_accuracy: 0.7735 - val_loss: 0.4805 - learning_rate: 0.0010
Epoch 5/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1137s 1s/step - accuracy: 0.7899 - loss: 0.4473 - val_accuracy: 0.7683 - val_loss: 0.4862 - learning_rate: 0.0010
Epoch 6/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1134s 1s/step - accuracy: 0.7989 - loss: 0.4319 - val_accuracy: 0.7729 - val_loss: 0.4867 - learning_rate: 5.0000e-04
Epoch 7/15
 979/1000 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accu

# Ergebnisse visualisieren

In [ ]:

fig_loss = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig_loss.suptitle('Loss', fontsize=20)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="upper left")
plt.show()

In [ ]:

fig_accuracy = plt.figure()
plt.plot(history.history['accuracy'], color='blue', label='accuracy')
plt.plot(history.history['val_accuracy'], color='green', label='val_accuracy')
fig_accuracy.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

# Evaluierung des Models anhand der Testdaten

In [ ]:
eval = model.evaluate(X_test,Y_test)

In [ ]:
print(eval)

### Speichern des Models

In [ ]:
model.save('testing4.keras')

# Explainable AI (LIME)
LIME ist ein Ansatz zur Erklärbarkeit von Modellen (Explainable AI, XAI), der entwickelt wurde, um Vorhersagen von Machine-Learning-Modellen interpretierbar und nachvollziehbar zu machen. Es bietet verständliche, lokale Erklärungen für individuelle Vorhersagen, unabhängig davon, wie komplex oder undurchsichtig das zugrunde liegende Modell ist.

### LimeTextExplainer
LimeTextExplainer ist eine spezialisierte Implementierung von LIME für die Verarbeitung von Textdaten. Es erklärt, welche Wörter oder Phrasen in einem Text am stärksten zur Vorhersage eines Modells beigetragen haben.

In [ ]:
import lime
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=['negative', 'positive'])
model_lime = tensorflow.keras.models.load_model('testing1.keras')
print(model.inputs)
print(model.outputs)


In [ ]:
def preprocess_texts(texts):
    cleaned_texts = []
    for text in texts:
        text = short_conv(text)
        text = text.lower()
        text = cleaning_stopwords(text)
        text = re.sub(userPattern,'', text)
        text = re.sub(urlPattern,'', text)
        text = re.sub(alphaPattern,' ', text)
        text = cleaning_punctuations(text)
        text = cleaning_repeating_char(text)
        text = tokenizer.tokenize(text)
        text = lemmatizer_on_text(text)
        cleaned_texts.append(text)
    tokenized_texts = tok.texts_to_sequences(cleaned_texts)
    padded_texts = sequence.pad_sequences(tokenized_texts, maxlen=500)
    del cleaned_texts
    return padded_texts

def predict_proba(texts):
    processed_texts = preprocess_texts(texts)  # Stellen Sie sicher, dass Texte verarbeitet werden
    probabilities = model_lime.predict(processed_texts)  # Gibt eine Wahrscheinlichkeit pro Text
    # Ergänzen der Wahrscheinlichkeit für die andere Klasse
    probabilities = np.hstack((1 - probabilities, probabilities))
    return probabilities




In [ ]:
example_text = "This product is insane, keep the good work up @Logitech. I LOVE IT"

In [ ]:
explanation = explainer.explain_instance(
    example_text,
    predict_proba,
    num_features=10
)

In [ ]:
explanation.show_in_notebook()